In [1]:
import random
from functools import reduce
from tqdm import tqdm

import torch
from transformers import LlamaForCausalLM, LlamaTokenizerFast, AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset, load_dataset
from torch.utils.data import DataLoader, RandomSampler
import pandas as pd

/data/dangnguyen/weak-to-strong/wts_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data/dangnguyen/weak-to-strong/wts_env/lib/python3.8/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
cache_path = '/data/dangnguyen/cache/'

In [3]:
model_name = 'sharpbai/Llama-2-7b-chat'
tokenizer = LlamaTokenizerFast.from_pretrained(model_name, cache_dir=cache_path)
model = LlamaForCausalLM.from_pretrained(model_name, cache_dir=cache_path)

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:04<00:00,  7.11it/s]
/data/dangnguyen/weak-to-strong/wts_env/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/data/dangnguyen/weak-to-strong/wts_env/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the genera

In [ ]:
model_name = 'mistralai/Mistral-7B-Instruct-v0.2'
model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir=cache_path)
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_path)

In [4]:
model.eval()
model.to('cuda')

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
 

### DAIR-AI Emotion dataset

In [ ]:
def map_label(label):
    if label == 0:
        return 'sadness'
    elif label == 1:
        return 'joy'
    elif label == 2:
        return 'love'
    elif label == 3:
        return 'anger'
    elif label == 4:
        return 'fear'
    elif label == 5:
        return 'surprise'
    else:
        return 'none'

In [ ]:
dataset = load_dataset('dair-ai/emotion', split='test')

# Filtering for obvious samples
keywords = ['sad', 'joy', 'love', 'anger', 'fear', 'surprise']
dataset_clean = {
    'text': [],
    'label': [],
}

for data in dataset:
    keep = True
    for kw in keywords:
        if kw in data['text']:
            keep = False
            break
    if keep:
        dataset_clean['text'].append(data['text'])
        dataset_clean['label'].append(data['label'])

In [ ]:
labels_eng = list(map(map_label, dataset_clean['label']))
dataset_clean['label_text'] = labels_eng

In [ ]:
dataset_torch = Dataset.from_dict(dataset_clean)
dataset_torch

In [ ]:
random.choice(dataset_torch)

In [ ]:
template = open('/data/dangnguyen/weak-to-strong/prompts/emotion_fewshot.txt').read()
template

### Tweet Sentiment dataset

In [ ]:
dataset = load_dataset('mteb/tweet_sentiment_extraction', split='test')
dataset

In [ ]:
dataset_torch = Dataset.from_dict(dataset[:2000])
dataset_torch

In [ ]:
random.choice(dataset_torch)

In [ ]:
template = open('/data/dangnguyen/weak-to-strong/prompts/tweet_sentiment_fewshot.txt').read()
template

### Running the model

In [ ]:
outputs = []
for sample in tqdm(dataset_torch):
    prompt = template.format(input=sample['text'])
    input_ids = tokenizer(prompt, return_tensors='pt').to('cuda')
    max_len = input_ids['input_ids'].shape[1]
    with torch.no_grad():
        output_ids = model.generate(input_ids['input_ids'], max_length=max_len+5)
    output = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    outputs.append(output[0])

In [ ]:
outputs

In [ ]:
output_labels = list(map(lambda x: x.split('>\n\nAnswer:\n')[1].lower().strip(), outputs))
output_labels

In [ ]:
# control
labels = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
control_labels = [random.choice(labels) for _ in range(2000)]

In [ ]:
correct = 0
for pred, gt in zip(output_labels, dataset_torch['label_text']):
    if pred == gt:
        correct += 1
        
accuracy = correct / len(output_labels)
accuracy

### Synthetic datasets: hierarchical equality

In [11]:
# Generating the data

def get_accuracy(preds, labels):
    num_correct = 0
    for pred, label in zip(preds, labels):
        if pred == label:
            num_correct += 1
    return num_correct / len(labels)

def get_equiv_rel_data(n_samples=1000, start=0, end=99):
    def sample_one():
        label = random.choice(['Yes', 'No'])
        if label == 'Yes':
            a, b = sample_equiv(start, end)
        else:
            a, b = sample_not_equiv(start, end)
        return (a, b, label)
    
    sampled_data = [sample_one() for _ in range(n_samples)]
    sampled_input = ["{} {}".format(a, b) for a, b, _ in sampled_data]
    sampled_label = [label for _, _, label in sampled_data]
    return (sampled_input, sampled_label)

def get_hierarch_data(n_samples=1000):
    def sample_one():
        values = ['Yes', 'No']
        var1 = random.choice(values)
        var2 = random.choice(values)
        if var1 == var2:
            label = 'Yes'
        else:
            label = 'No'
            
        if var1 == 'Yes':
            a, c = sample_equiv()
        else:
            a, c = sample_not_equiv()
        if var2 == 'Yes':
            b, d = sample_equiv()
        else:
            b, d = sample_not_equiv()
        return (a, b, c, d, label)
    
    sampled_data = [sample_one() for _ in range(n_samples)]
    sampled_input = ["({},{}) ({},{})".format(a, b, c, d) \
                     for a, b, c, d, _ in sampled_data]
    sampled_label = [label for _, _, _, _, label in sampled_data]
    return (sampled_input, sampled_label)

def sample_equiv(start=0, end=99):
    b = random.randint(start, end-3)
    a = b + 3
    return a, b

def sample_not_equiv(start=0, end=99):
    a = random.randint(start, end)
    b = random.randint(start, end)
    while a - b == 3:
        b = random.randint(start, end)
    return a, b

In [ ]:
total_data_size = 1000
# raw_data = get_equiv_rel_data(total_data_size, start=50, end=99)
raw_data = get_hierarch_data(total_data_size)

In [ ]:
num_pos = 0
for label in raw_data[1]:
    if label == 'Yes':
        num_pos += 1
num_pos / len(raw_data[1])

In [ ]:
template = open('/data/dangnguyen/weak-to-strong/weak-to-strong/prompts/hierarchical_equiv_zeroshot.txt').read()
template

In [ ]:
prompts = [template.format(INPUT=data) for data in raw_data[0]]
prompts

In [ ]:
with open('./datasets/hierarchical_equivalence/train.csv', 'w') as fw:
    fw.write('input,label\n')
    for prompt, label in zip(prompts, raw_data[1]):
        prompt_csv = prompt.replace('\n', '\\n')
        fw.write('"{}",{}\n'.format(prompt_csv, label))

In [5]:
# dataset = load_dataset('dangnguyen0420/equivalence_relation')
dataset = load_dataset('dangnguyen0420/hierarchical_equivalence')
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'label'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['input', 'label'],
        num_rows: 1000
    })
})

In [8]:
train_dataloader = DataLoader(dataset['train'], batch_size=16)
test_dataloader = DataLoader(dataset['test'], batch_size=16)

In [13]:
all_preds = []
all_labels = []

with torch.no_grad():
    epoch_iterator = tqdm(train_dataloader)
    for _, inputs in enumerate(epoch_iterator):
        input_ids = tokenizer(inputs['input'], padding=True, return_tensors='pt').to('cuda')
        output_ids = model(**input_ids)
        
        pred_ids = output_ids.logits[:, -1, :].argmax(dim=-1)
        preds = tokenizer.batch_decode(pred_ids)
        
        all_preds.append(preds)
        all_labels.append(inputs['label'])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [02:38<00:00,  2.52s/it]


In [14]:
all_preds = reduce(lambda x, y: x + y, all_preds)
all_labels = reduce(lambda x, y: x + y, all_labels)

test_acc = get_accuracy(all_preds, all_labels)
test_acc

0.498